# Milestone 2

## Introduction
Differentiation is used in many applications, such as finding stationary points of defined functions or minimizing objective loss functions in machine learning applications. 
But differentiating an arbitrary function &#8477;<sup>n</sup> &#8594; &#8477;<sup>m</sup> is generally not an easy task. 
When the function can be expressed as a composition of differentiable elementary functions (which in most cases is true), Automatic Differentiation (AD) can help. 
AD has become one of the most popular techniques for finding derivatives and is often preferred over symbolic differentation and numerical differentiation because of its efficiency and stability.

 
## Background

How does AD do it? AD takes an input function and breaks it down into a set of elementary functions. It uses symbolic differentiation of the elementary functions to calculate their derivatives at specific values. These elementary functions are then combined using common mathematical functions, such as addition or multiplication. The derivatives of the combined functions (which we will refer to as nodes) can be found using the chain rule and the derivatives calculated from the earlier elementary functions. This process repeats (as more complex functions are combined) until the full functions derivative has been calculated. As an example, if you want to compute the derivative of *sin(tan(xy) + cos(x + y))* you can first compute the derivatives w.r.t *x* and *y* of *tan(xy)* and *cos(x + y)*, then add those together, and then get the derivative of the entire function using the chain rule. 

There are two common methods for implementing AD, forward mode and backward mode (of which the popular backpropagation algorithm for neural networks is a special case). Given a function &#8477;<sup>n</sup> &#8594; &#8477;<sup>m</sup>, forward mode fixes the independent variables or input (n) to solve for m outputs; whereas backward mode fixes the dependent variables or outputs (m). This means that forward mode is more efficient when m>>n and backward mode is more efficient when n>>m. That's why backward mode is more common in "big data" problems where there are often far more features than data. 

The AD process can be represented using either the computational trace or the computation graph. The computational trace is a table that stores the elementary and intermediary functions, their derivatives, and the derivative values evaluated at an input (if provided). This process can also be visually representated using a computational graph, where nodes are rows in the computational trace and arrows show how nodes are combined and by what elementary operations. 

## How to Use funkyAD

The software funkyAD is a software package that the user will interact with using the AD class. This AD class allows the user to differentiate a specified function by wrapping it into an AD object, automatically differentiate it, access the results and inspect the intermediate steps (if desired). The package is intended for use by developers on personal computers, as a building block on top of which other functionality may be developed.

In [1]:
# import base from funkyAD
from funkyAD import base

ModuleNotFoundError: No module named 'functions'

### Basic examples of funkyAD

In [2]:
# show basic functionality of funkyAD
def f(x):
    return x**3

print(AD(f).grad(2))
print(AD(f).grad(3))

NameError: name 'AD' is not defined

In [ ]:
def f(x,y):
    return x+y

print(AD(f).grad(3,5))

### Example 1: Using funky AD for Newton's Method

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# instantiate an automatic differentiation object to be used in the forward mode.
def newtonroot(f, x):
    # find f(x) and f'(x) jacobian using funkyAD package 
    fx = f(x)
    dfx = AD(f).grad(x) # alternative this can be written grad(f)(x)
    x_next = x - fx/dfx 
    return x_next 

def get_root(f,x_start):
    delta = 10
    path = [x_start]
    x = x_start
    while delta > 1e-6:
        new_x = newtonroot(f,x) 
        path.append(new_x)
        delta = abs(x - new_x)
        x = new_x 
    return(x, path)

In [ ]:
# root method for scalar function (e.g. y = f(x))
def f(x):
    return x**2+x 

# try 1 initialization
zero1, path1 = get_root(f, 1)

# try a different initialization 
zero2, path2  = get_root(f, -2)

In [ ]:
xs = np.linspace(-2.5, 1.5, 100)
path1_ys = [0]*len(path1)
path2_ys = [0]*len(path2)


plt.plot(xs, f(xs))
plt.axhline(y=0, color='r', linestyle='-')
plt.scatter(path1, path1_ys)
plt.scatter(path2, path2_ys)
plt.title('Root finding for x^2 + x')
plt.show()

## Software Organization

#### Directory Structure: 
```
.
| README.md
| LICSENSE
| requirements.txt
| setup.py
| 
|--docs
|
|  | milestone1.md
|  | milestone2.ipynb
|  |-examples
|  | |- AD_users_intro.ipynb 
|
|--funkyAD
|  | __init__.py 
|  | base.py
|  | functions.py 
|  | helpers.py
|  |-tests
|
|--benchmarks
```

#### Modules (functionality)

The basic package will include a module for forward AD, which takes a function as an argument and can return any of the following: the derivative, the derivative evaluated at a given value, the trace, etc. We also hope to implement a backward AD module, which takes the same input and returns the same output as the forward AD module, but uses backpropagation to calculate the return values and is more efficient for single-output functions.

##### Feedback from milestone1 comments
Module explanation is very high-level; more detail about each module's functionality would be nice. 

#### Test suite
Our test suite will live within the funkyAD directory. We plan to use TravisCI, CodeCov, doctests and unittests for testing. We plan to use unittests for most of our testing (dottests only occasionally). We aim for as much code coverage as possible, and specifically target Exception raising and handling of edge cases.

We plan to write extensive and detailed docstrings for all functions that will be accessible to the user, and replicate those in a nicer format in the corresponding docs folder (e.g. the file /funkyAD/AD.py will have its docs in /funkyAD/AD.html). A nice collection of examples for installation and usage will hopefully help the user in getting a working knowledge of the library quickly.

#### Distribution
The funkyAD package will be distributed with PyPI. Consequently, users will be able to install the package using the ubiquitous pip package manager. It will follow the guidelines and instructions in the official Python documentation. Installation instructions

	conda create -n env_name python=3.6 anaconda
	source activate env_name
	pip install -r requirements.txt
	pip install -i https://test.pypi.org/simple/ funkyAD-funkyADers==0.0.3


#### Software packaging 
According to common practice, we will include \_\_init\_\_.py files and setup.py so that automated tools can install and set up the library properly. We will choose transparent file names so that import statements are intuitive to the end user.

It will be packaged as a Wheel for fast and easy installation.

## Implementation

#### Implementation (4/5)
1. You needed a more in-depth explanation of how you will deal with vector-valued functions.
2. How will you be storing "eval_trace" and "graph" in the AD class?

In funkyAD we define 3 main classes: AD, Node, and ElementaryFunction. AD is the class that the user will interact with. It takes in an arbitrary function from the user and calls the necessary functions and classes in order to calculate the gradient. It will have transparently named methods and syntactic sugar when appropriate. The user should not know how anything else in the library works for simple usage.

The Node class is essentially a row in our trace table, it has subclasses for input nodes (InputNode) and output nodes (OutputNodes). Nodes are connected and can be added or multiplied together to form new nodes, via the dunder methods\_\_add\_\_ etc, which allows us to build up the trace table. Dual numbers are essentially encoded as the pair (Node.val, Node.grad_val) in our implementation, and additions/operation are performed when calling ElementaryFunction(Node1, Node2). A Node is, essentially, an extension of the Box class in HW4. If the user has defined a function f, we will call f(InputNode(input)) and the successive operation performed on that node will allow us to recover the evaluation trace.

The ElementaryFunction Class defines the functions and derivatives of elementary functions passed in by the user, such as sin, log, etc. We also allow the user to add their own elementary function to the list if we do not include the elementary function they need in the initial library list. An appropriate exception is raised if the user tries to utilize a function that is not defined as an instance of the ElementaryFunction class.

```
# External dependencies
import numpy
import doctest
import unitest
import pytest

# the AD class instigates the differentation process and stores all output values
class AD():

Methods:
 
def __init__(): initialize AD object 
def _buildgraph(): build the evaluation graph
def gradient(forward): run forward AD, allows for option for backward AD if implemented
def _createnodes(): create notes for the evaluation trace
def set_seed(): set the seed (by default is the identity matrix)
def get_seed(): return the seed
def eval_trace: return evaluation trace  
def print_graph(): return the evaluation graph   

Attributes: 

nodeList: list of nodes
inputNodeList: the list of inputs
outputNodelist: the list of outputs
eval_trace: the evaluation trace   
graph: the evaluation graph 
history: history of calls memoized for increased performance
(dunder methods e.g. __str__ redefined)

# Node classes and subclasses 
class Node(): the node class stores relevant information for each node in the graph
class InputNode(Node): a subclass of the Node class
class OutputNode(Node): a subclass of the Node class

Methods:
__add__  (__radd__): add two nodes
__mult__ (__rmult__): multiple two nodes
(other dudner methods redefined as appropriate)

Attributes:   
val: the value of the node   
gradient_val: the gradient of the node
parents: list of parent nodes
children: list of children nodes
el: elementary function that the node computes (an ElementaryFunction object)
SHOULD THIS BE STORED AS A DUAL NUMBER?

# The Elementary Function stores elementary functions and their derivatives
class ElementaryFunction(): 

Methods:
__call__(): returns the output of the function
derivative(): returns the derivative for a given input

Attributes: 
ninputs: number of inputs  
noutputs: number of outputs 


```

## Future Features